In [ ]:
Ordinal Regression ( also known as Ordinal Logistic Regression) is another extension of binomial logistics regression. Ordinal regression is used to predict the dependent variable with ‘ordered’ multiple categories and independent variables. In other words, it is used to facilitate the interaction of dependent variables (having multiple ordered levels) with one or more independent variables.



In [1]:
library(dplyr)
library(package = feather)
library(pscl)
library(MASS)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Classes and Methods for R developed in the
Political Science Computational Laboratory
Department of Political Science
Stanford University
Simon Jackman
hurdle and zeroinfl functions by Achim Zeileis

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select



In [2]:
df <- read_feather('merged_data.feather')


Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”

In [3]:
# Description을 제외하고 필터링
df_trim = dplyr::select(df, -Description, -PetID, -RescuerID, -Name)
# is_train이 True인 데이터 열만 추려냅니다.
df_trim = dplyr::filter(df_trim, is_train)
# 각각의 컬럼을 categorical 형으로 바꾸줍니다.
df_trim$Dewormed = as.factor(df_trim$Dewormed)
df_trim$Gender = as.factor(df_trim$Gender)
df_trim$Vaccinated = as.factor(df_trim$Vaccinated)
df_trim$Sterilized = as.factor(df_trim$Sterilized)
df_trim$Type = as.factor(df_trim$Type)
df_trim$VideoAmt = as.factor(df_trim$VideoAmt)
df_trim$ColorName1 = as.factor(df_trim$ColorName1)
df_trim$ColorName2 = as.factor(df_trim$ColorName2)
df_trim$ColorName3 = as.factor(df_trim$ColorName3)
df_trim$BreedName1 = as.factor(df_trim$BreedName1)
df_trim$BreedName2 = as.factor(df_trim$BreedName2)
df_trim$StateName = as.factor(df_trim$StateName)

In [4]:
# Health와 MaturitySize는 ordinal value이기때문에 그에 맞게 변환해줍니다.
df_trim$Health <- ordered(df_trim$Health, levels = 1:3,
                              labels = c("Good", "Soso", "Bad"))

df_trim$MaturitySize <- ordered(df_trim$MaturitySize, levels = 1:4,
                              labels = c("Small", "Medium", "Large", "Very large"))


In [5]:
df_trim$Fee_free <- df_trim$Fee == 0


In [6]:
df_trim$AdoptionSpeed <- as.ordered(df_trim$AdoptionSpeed)

In [7]:
head(df_trim)

AdoptionSpeed,Age,Dewormed,Fee,FurLength,Gender,Health,MaturitySize,PhotoAmt,Quantity,⋯,Vaccinated,VideoAmt,is_train,ColorName1,ColorName2,ColorName3,BreedName1,BreedName2,StateName,Fee_free
2,3,No,100,1,Male,Good,Small,1,1,⋯,No,0,TRUE,Black,White,NA,Tabby,NA,Selangor,FALSE
0,1,Not Sure,0,2,Male,Good,Medium,2,1,⋯,Not Sure,0,TRUE,Black,Brown,NA,Domestic Medium Hair,NA,Kuala Lumpur,TRUE
3,1,Yes,0,2,Male,Good,Medium,7,1,⋯,Yes,0,TRUE,Brown,White,NA,Mixed Breed,NA,Selangor,TRUE
2,4,Yes,150,1,Female,Good,Medium,8,1,⋯,Yes,0,TRUE,Black,Brown,NA,Mixed Breed,NA,Kuala Lumpur,FALSE
2,1,No,0,1,Male,Good,Medium,3,1,⋯,No,0,TRUE,Black,NA,NA,Mixed Breed,NA,Selangor,TRUE
2,3,No,0,1,Female,Good,Medium,2,1,⋯,No,0,TRUE,Cream,Gray,NA,Domestic Short Hair,NA,Selangor,TRUE


In [8]:
model1 <- polr(AdoptionSpeed ~ Age, data = df_trim, Hess=TRUE)


In [9]:
summary(model1)

Call:
polr(formula = AdoptionSpeed ~ Age, data = df_trim, Hess = TRUE)

Coefficients:
      Value Std. Error t value
Age 0.01055  0.0008457   12.48

Intercepts:
    Value    Std. Error t value 
0|1  -3.4776   0.0506   -68.7466
1|2  -1.0915   0.0207   -52.6301
2|3   0.1159   0.0183     6.3148
3|4   1.0591   0.0204    51.8569

Residual Deviance: 43776.85 
AIC: 43786.85 

In [10]:
ctable <-coef(summary(model1))
p <- pnorm(abs(ctable[, "t value"]), lower.tail=FALSE) *2

In [11]:
ctable <- cbind(ctable, "p value" = p)
ctable

,Value,Std. Error,t value,p value
Age,0.0105532,0.0008457408,12.478059,9.835463e-36
0|1,-3.4775805,0.0505855205,-68.746559,0.000000e+00
1|2,-1.0915089,0.0207392659,-52.630062,0.000000e+00
2|3,0.1158640,0.0183480908,6.314774,2.705569e-10
3|4,1.0591175,0.0204238517,51.856891,0.000000e+00


In [18]:
belta_age = 10


In [19]:
y = 0.01055 * belta_age


In [20]:
p1 = 1 / (1 + exp(-(-3.4776 - y)))
p12 = 1 / (1 + exp(-(-1.0915 - y)))
p23 = 1 / (1 + exp(-(0.1159 - y)))
p34 = 1 / (1 + exp(-(1.0591 - y)))


In [21]:
p2 = p12 - p1
p3 = p23 - p12
p4 = p34 - p23
p5 = 1- p34




In [17]:
c(p1, p2, p3, p4, p5) # belta_age = 20


[1] 0.02439689 0.18934768 0.26249833 0.22392552 0.29983158

In [22]:
c(p1, p2, p3, p4, p5) # belta_age = 10



[1] 0.02703805 0.20497128 0.27059065 0.21923861 0.27816141

In [23]:
model2 <- polr(AdoptionSpeed ~ MaturitySize + Type + Fee_free, data = df_trim, Hess=TRUE)


In [24]:
summary(model2)
ctable <- coef(summary(model2))
p <- pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2
ctable <- cbind(ctable, "p value" = p)
ctable


Call:
polr(formula = AdoptionSpeed ~ MaturitySize + Type + Fee_free, 
    data = df_trim, Hess = TRUE)

Coefficients:
                  Value Std. Error t value
MaturitySize.L -0.29004    0.20295 -1.4291
MaturitySize.Q -0.35361    0.15332 -2.3063
MaturitySize.C  0.04233    0.07675  0.5515
TypeDog         0.28811    0.03009  9.5736
Fee_freeTRUE   -0.11614    0.04075 -2.8501

Intercepts:
    Value    Std. Error t value 
0|1  -3.3382   0.0975   -34.2421
1|2  -0.9432   0.0861   -10.9594
2|3   0.2694   0.0857     3.1432
3|4   1.2101   0.0862    14.0333

Residual Deviance: 43751.32 
AIC: 43769.32 

,Value,Std. Error,t value,p value
MaturitySize.L,-0.29003883,0.20295220,-1.4290992,1.529757e-01
MaturitySize.Q,-0.35360970,0.15332012,-2.3063490,2.109114e-02
MaturitySize.C,0.04232535,0.07675111,0.5514624,5.813168e-01
TypeDog,0.28811483,0.03009464,9.5736271,1.032195e-21
Fee_freeTRUE,-0.11613777,0.04074934,-2.8500528,4.371197e-03
0|1,-3.33818490,0.09748787,-34.2420552,5.726468e-257
1|2,-0.94324774,0.08606749,-10.9593962,5.989763e-28
2|3,0.26938860,0.08570405,3.1432424,1.670874e-03
3|4,1.21011864,0.08623217,14.0332615,9.755963e-45


In [26]:
model3 <- polr(AdoptionSpeed ~ MaturitySize + Type + Fee_free + MaturitySize:Fee_free, data=df_trim, Hess=TRUE)

In [27]:
summary(model3)
ctable <- coef(summary(model3))
p <- pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2
ctable <- cbind(ctable, "p value" = p)
ctable

Call:
polr(formula = AdoptionSpeed ~ MaturitySize + Type + Fee_free + 
    MaturitySize:Fee_free, data = df_trim, Hess = TRUE)

Coefficients:
                               Value Std. Error t value
MaturitySize.L              -0.46457    0.48960 -0.9489
MaturitySize.Q              -0.31439    0.36912 -0.8517
MaturitySize.C               0.11282    0.18112  0.6229
TypeDog                      0.28586    0.03023  9.4562
Fee_freeTRUE                -0.04256    0.20284 -0.2098
MaturitySize.L:Fee_freeTRUE  0.21310    0.53776  0.3963
MaturitySize.Q:Fee_freeTRUE -0.05341    0.40571 -0.1317
MaturitySize.C:Fee_freeTRUE -0.09403    0.19995 -0.4703

Intercepts:
    Value    Std. Error t value 
0|1  -3.2822   0.1912   -17.1645
1|2  -0.8871   0.1857    -4.7772
2|3   0.3258   0.1856     1.7556
3|4   1.2667   0.1858     6.8160

Residual Deviance: 43747.57 
AIC: 43771.57 

,Value,Std. Error,t value,p value
MaturitySize.L,-0.46457193,0.48959732,-0.9488858,3.426787e-01
MaturitySize.Q,-0.31439429,0.36912373,-0.8517315,3.943632e-01
MaturitySize.C,0.11282068,0.18111706,0.6229158,5.333399e-01
TypeDog,0.28586454,0.03023032,9.4562194,3.192785e-21
Fee_freeTRUE,-0.04256437,0.20283707,-0.2098451,8.337886e-01
MaturitySize.L:Fee_freeTRUE,0.21310134,0.53776199,0.3962744,6.919026e-01
MaturitySize.Q:Fee_freeTRUE,-0.05341253,0.40571353,-0.1316509,8.952605e-01
MaturitySize.C:Fee_freeTRUE,-0.09403073,0.19995433,-0.4702610,6.381685e-01
0|1,-3.28219867,0.19122022,-17.1644955,4.897201e-66
1|2,-0.88707839,0.18568947,-4.7772144,1.777403e-06


In [28]:
model4 <- polr(AdoptionSpeed ~ Gender + Sterilized + Type, data = df_trim, Hess=TRUE)


In [29]:
summary(model4)
ctable <- coef(summary(model4))
p <- pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2
ctable <- cbind(ctable, "p value" = p)
ctable


Call:
polr(formula = AdoptionSpeed ~ Gender + Sterilized + Type, data = df_trim, 
    Hess = TRUE)

Coefficients:
                     Value Std. Error t value
GenderMale         -0.1915    0.03210  -5.964
GenderMixed         0.1974    0.04472   4.414
SterilizedNot Sure  0.5822    0.04833  12.047
SterilizedYes       0.8632    0.03809  22.661
TypeDog             0.2894    0.02986   9.693

Intercepts:
    Value    Std. Error t value 
0|1  -3.2850   0.0555   -59.2219
1|2  -0.8775   0.0312   -28.1240
2|3   0.3640   0.0303    12.0200
3|4   1.3396   0.0322    41.6658

Residual Deviance: 43165.48 
AIC: 43183.48 

,Value,Std. Error,t value,p value
GenderMale,-0.1914758,0.03210365,-5.964300,2.456849e-09
GenderMixed,0.1974077,0.04472340,4.413969,1.014925e-05
SterilizedNot Sure,0.5821836,0.04832520,12.047205,2.006374e-33
SterilizedYes,0.8632177,0.03809268,22.660985,1.087223e-113
TypeDog,0.2893794,0.02985589,9.692539,3.243628e-22
0|1,-3.2850448,0.05547006,-59.221947,0.000000e+00
1|2,-0.8774937,0.03120085,-28.124037,4.979921e-174
2|3,0.3640102,0.03028375,12.019983,2.790285e-33
3|4,1.3396135,0.03215139,41.665808,0.000000e+00


In [30]:
model5 <- polr(AdoptionSpeed ~ Sterilized + Type + Sterilized:Type, data = df_trim, Hess=TRUE)


In [31]:
summary(model5)
ctable <- coef(summary(model5))
p <- pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2
ctable <- cbind(ctable, "p value" = p)
ctable


Call:
polr(formula = AdoptionSpeed ~ Sterilized + Type + Sterilized:Type, 
    data = df_trim, Hess = TRUE)

Coefficients:
                             Value Std. Error t value
SterilizedNot Sure          0.5133    0.07734   6.637
SterilizedYes               0.9335    0.05842  15.980
TypeDog                     0.2931    0.03531   8.299
SterilizedNot Sure:TypeDog  0.1082    0.09885   1.095
SterilizedYes:TypeDog      -0.1518    0.07587  -2.001

Intercepts:
    Value    Std. Error t value 
0|1  -3.2379   0.0534   -60.6059
1|2  -0.8332   0.0277   -30.1200
2|3   0.4037   0.0269    15.0184
3|4   1.3754   0.0290    47.4094

Residual Deviance: 43239.50 
AIC: 43257.50 

,Value,Std. Error,t value,p value
SterilizedNot Sure,0.5133009,0.07734026,6.636917,3.203129e-11
SterilizedYes,0.9334567,0.05841546,15.979616,1.772338e-57
TypeDog,0.2930830,0.03531419,8.299300,1.047269e-16
SterilizedNot Sure:TypeDog,0.1082439,0.09885414,1.094986,2.735230e-01
SterilizedYes:TypeDog,-0.1518280,0.07587213,-2.001103,4.538124e-02
0|1,-3.2379195,0.05342578,-60.605932,0.000000e+00
1|2,-0.8331554,0.02766118,-30.120017,2.650197e-199
2|3,0.4036601,0.02687761,15.018450,5.559264e-51
3|4,1.3753874,0.02901088,47.409365,0.000000e+00


In [32]:
model6 <- polr(AdoptionSpeed ~ Sterilized + Type + Gender + Sterilized:Type + Sterilized:Gender + Type:Gender, data = df_trim, Hess=TRUE)


In [33]:
summary(model6)
ctable <- coef(summary(model6))
p <- pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2
ctable <- cbind(ctable, "p value" = p)
ctable


Call:
polr(formula = AdoptionSpeed ~ Sterilized + Type + Gender + Sterilized:Type + 
    Sterilized:Gender + Type:Gender, data = df_trim, Hess = TRUE)

Coefficients:
                                  Value Std. Error t value
SterilizedNot Sure              0.43066    0.09493  4.5365
SterilizedYes                   0.96413    0.06967 13.8390
TypeDog                         0.37306    0.04807  7.7614
GenderMale                     -0.09879    0.05254 -1.8801
GenderMixed                     0.17742    0.06458  2.7473
SterilizedNot Sure:TypeDog      0.13233    0.09942  1.3310
SterilizedYes:TypeDog          -0.16812    0.07695 -2.1849
SterilizedNot Sure:GenderMale   0.04270    0.10611  0.4024
SterilizedYes:GenderMale       -0.01935    0.07990 -0.2422
SterilizedNot Sure:GenderMixed  0.31968    0.13931  2.2947
SterilizedYes:GenderMixed       0.03464    0.15311  0.2263
TypeDog:GenderMale             -0.16690    0.06491 -2.5711
TypeDog:GenderMixed            -0.01763    0.09067 -0.1945

Interce

,Value,Std. Error,t value,p value
SterilizedNot Sure,0.43065964,0.09493117,4.5365463,5.718293e-06
SterilizedYes,0.96413202,0.06966752,13.8390470,1.481578e-43
TypeDog,0.37305776,0.04806581,7.7613956,8.399987e-15
GenderMale,-0.09879204,0.05254480,-1.8801488,6.008780e-02
GenderMixed,0.17742332,0.06458066,2.7473134,6.008569e-03
SterilizedNot Sure:TypeDog,0.13233320,0.09942182,1.3310277,1.831799e-01
SterilizedYes:TypeDog,-0.16811790,0.07694507,-2.1849081,2.889559e-02
SterilizedNot Sure:GenderMale,0.04270172,0.10610777,0.4024372,6.873623e-01
SterilizedYes:GenderMale,-0.01935089,0.07989857,-0.2421932,8.086304e-01
SterilizedNot Sure:GenderMixed,0.31967671,0.13931340,2.2946586,2.175269e-02


In [34]:
model7 <- polr(AdoptionSpeed ~ StateName + BreedName1 + FurLength + Sterilized + Type + Gender + Fee_free + MaturitySize + PhotoAmt + Vaccinated + VideoAmt + Health, data = df_trim %>% dplyr::filter(is_train) %>% dplyr::select(-is_train), Hess=TRUE)


Warning message in polr(AdoptionSpeed ~ StateName + BreedName1 + FurLength + Sterilized + :
“design appears to be rank-deficient, so dropping some coefs”

In [35]:
summary(model7)
ctable <- coef(summary(model7))
p <- pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2
ctable <- cbind(ctable, "p value" = p)
ctable

Call:
polr(formula = AdoptionSpeed ~ StateName + BreedName1 + FurLength + 
    Sterilized + Type + Gender + Fee_free + MaturitySize + PhotoAmt + 
    Vaccinated + VideoAmt + Health, data = df_trim %>% dplyr::filter(is_train) %>% 
    dplyr::select(-is_train), Hess = TRUE)

Coefficients:
                                                             Value Std. Error
StateNameKedah                                            0.817773  1.910e-01
StateNameKelantan                                         0.316012  5.284e-01
StateNameKuala Lumpur                                     0.328446  8.764e-02
StateNameLabuan                                           1.231552  9.678e-01
StateNameMelaka                                           1.197249  1.930e-01
StateNameNegeri Sembilan                                  0.732248  1.442e-01
StateNamePahang                                           0.018611  2.325e-01
StateNamePerak                                            0.577114  1.214e-01
StateNameP

,Value,Std. Error,t value,p value
StateNameKedah,0.81777321,1.910075e-01,4.281366e+00,1.857492e-05
StateNameKelantan,0.31601152,5.284448e-01,5.980029e-01,5.498380e-01
StateNameKuala Lumpur,0.32844562,8.763892e-02,3.747714e+00,1.784534e-04
StateNameLabuan,1.23155240,9.678261e-01,1.272494e+00,2.031978e-01
StateNameMelaka,1.19724856,1.930498e-01,6.201759e+00,5.583561e-10
StateNameNegeri Sembilan,0.73224775,1.442202e-01,5.077291e+00,3.828545e-07
StateNamePahang,0.01861072,2.324770e-01,8.005401e-02,9.361943e-01
StateNamePerak,0.57711391,1.213656e-01,4.755170e+00,1.982797e-06
StateNamePulau Pinang,0.57948202,1.037699e-01,5.584297e+00,2.346472e-08
StateNameSabah,0.33452074,4.576638e-01,7.309312e-01,4.648212e-01
